In [ ]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import schema
import math

In [ ]:
signal_params = [
    {
        'freq_hz': 0.15,
        'ampl': 0.837,
        'phase_rad': 0.0
    },
    {
        'freq_hz': 0.2,
        'ampl': 0.286,
        'phase_rad': 0.0
    },
    {
        'freq_hz': 0.12,
        'ampl': 0.227,
        'phase_rad': 0.0
    },
    {
        'freq_hz': 0.06,
        'ampl': 0.183,
        'phase_rad': 0.0
    }
]

display_freq_limit = (0.025, 1)

Comparing tests and expected

In [ ]:
#This gives us a new sampling method which mimics the actual scheduling algorithm 
#These are the intervals at which we will sample the given signals (signal_params)

json_file_actual = open("../tests/no_check_outputs/actual_file_tests.log")

#how long does it run plus one period
maxVal = 0

scheduleA = np.array([])
scheduleB = np.array([])
scheduleC = np.array([])

for line in json_file_actual:
    if(line.find("test") != -1): #get the line we want to measure
        startInd = int(line.find("["))+1
        endInd = int(len(line)-2)
    
        values = line[startInd:endInd] #get the lists without the end brackets
        tasks = values.split(',')
        for task in tasks:
            actual = task[task.find("\"")+1:len(task)-2] #get rid of the quotation marks
            times = actual.split('|')
            label = times[0]
            start = float(times[1])/1000
            end = float(times[1])/1000
            if label == 'A':
                scheduleA = np.append(scheduleA, start)
            elif label == 'B':
                scheduleB = np.append(scheduleB, start)
            elif label == 'C':
                scheduleC = np.append(scheduleC, start)
            else:
                print("Error, No Acceptable Label.\n")
            if maxVal < end:
                maxVal = end
        break

# print(scheduleA)
# print(scheduleB)

Input Sampling Patterns

In [ ]:
#temp 
delay_injection = 10
delay_amplitude = 0.1

sampling_freq_hz: float = 1000/25 #40 hz period, 25 ms period

sampling_duration_s = 900

sampling_int = 1.0 / sampling_freq_hz

startTime = scheduleA[0]

Process Sampling Patterns

In [ ]:


#The following code processes the inputs and creates an ideal sampling pattern
#It also introduces various types of noise to compare the efficiency 
#selects evenly spaced intervals when the samples are actually taken


#Reference sampling number must match the number of given samples in the schedule being compared
#Need to figure out how to compare multiple 
reference_sampling = np.arange(startTime, sampling_duration_s + sampling_int, sampling_int, dtype=np.float64)


#total number of samples taken
n_samples = len(scheduleA)

print(len(reference_sampling))
print(len(scheduleA))

temp_interval = len(np.heaviside(np.arange(-1 * delay_injection, sampling_duration_s - delay_injection + sampling_int, sampling_int), 1) * delay_amplitude)

if len(scheduleA) > temp_interval:
    scheduleA = scheduleA[:temp_interval]
elif temp_interval > len(scheduleA):
    interval_priority_sampling = (np.heaviside(np.arange(-1 * delay_injection, sampling_duration_s - delay_injection + sampling_int, sampling_int), 1) * delay_amplitude)
    interval_priority_sampling = interval_priority_sampling[:len(scheduleA)] + scheduleA
else:
    interval_priority_sampling = scheduleA + np.heaviside(np.arange(-1 * delay_injection, sampling_duration_s - delay_injection + sampling_int, sampling_int), 1) * delay_amplitude

# increment_priority_sampling = reference_sampling.copy()
# increment_priority_sampling[delay_injection] += delay_amplitude

gaussian_component_noise = scheduleA + np.random.normal(0, 0.0088,n_samples)

combined_noise = scheduleA + gaussian_component_noise - scheduleA + interval_priority_sampling

sampling_behaviors = {
    'Ideal_Timing': reference_sampling,
    # 'Interval': interval_priority_sampling,
    'Actual_Timing': scheduleA,
    'Gaussian Noise': gaussian_component_noise,
    'Everything': combined_noise
}

In [ ]:
class SigGen:
    #Validates the data -> are the numbers entered actualy floats, etc.
    def __init__(self, params: List[Dict[str, float]]):
        signal_params_schema = schema.Schema(
            [
                {
                    'freq_hz': schema.Or(int, float),
                    'ampl': schema.Or(int, float),
                    'phase_rad': schema.Or(int, float)
                }
            ]
        )
        signal_params_schema.validate(params)
        self._params = params
    
    def add_signal(self, freq_hz: float, ampl: float, phase_rad: float):
        print("Signal added...\n")
        self._params.append({
            'freq_hz': freq_hz,
            'ampl': ampl,
            'phase_rad': phase_rad
        })
    

    #Generate the height of each signal at a given time?
    def generate(self, t: np.ndarray) -> np.ndarray:
        output = np.zeros(t.shape, t.dtype)
        for component in signal_params:
            output += component['ampl'] * np.sin(t * 2 * np.pi * component['freq_hz'] + component['phase_rad'])
        return output
    
    def __call__(self, t: np.ndarray) -> np.ndarray:
        return self.generate(t)

sig_gen = SigGen(signal_params)

In [ ]:
reference_signal = sig_gen(reference_sampling)
reference_spectrum = np.fft.rfft(reference_signal)

signals = {label:sig_gen(sampling_sequence)
          for label, sampling_sequence in sampling_behaviors.items()}
spectra = {label:np.fft.rfft(signal) for label, signal in signals.items()}
fft_freq = {label:np.fft.rfftfreq(len(timing), sampling_int)
            for label, timing in sampling_behaviors.items()}

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)


for label, sequence in sampling_behaviors.items():
    ax.scatter(np.arange(len(sequence)), sequence - reference_sampling[:len(sequence)], label=label)

ax.legend()
ax.set_ylabel('Sampling Error')
ax.set_xlabel('Sample Index')
ax.grid()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(reference_sampling, reference_signal)
ax.grid()

ax.set_ylabel('Signal Amplitude')
ax.set_xlabel('Time (s)')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

for label, signal in signals.items():
    ax.plot(np.arange(len(signal)), signal, label=label)

ax.grid()
ax.set_ylabel('Signal Amplitude')
ax.set_xlabel('Sample Index')
ax.legend()

In [ ]:
#Generate amplitude vs. time index (Instead of amplitude vs. sample)

# fig = plt.figure()
# fig.add_subplot(111)

# for label, signal in signals.items():
#     ax.plot(scheduleA, signal[:len(scheduleA)],label=label)


# ax.grid()
# ax.set_ylabel('Signal Amplitude (m)')
# ax.set_xlabel('Time Sample Start (s)')
# ax.legend()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

for label, signal in signals.items():
    ax.plot(np.arange(len(signal)), signal - reference_signal[:len(signal)], label=label)

ax.grid()
ax.set_ylabel('Signal Difference')
ax.set_xlabel('Sample Index')
ax.legend()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)

labels = []

for label in spectra:
    labels.append(label)
    ax1.plot(fft_freq[label], 2 / n_samples * np.abs(spectra[label]), label=label)
    ax2.plot(fft_freq[label], np.angle(spectra[label]))


print(labels)

ax1.grid()
ax2.grid()
ax1.set_ylabel('Amplitude')
ax2.set_ylabel('Phase (rad)')
ax2.set_xlabel('Frequency (Hz)')
ax1.legend()
ax1.semilogy()
ax1.set_ylim(1e-6)
ax1.set_xlim(display_freq_limit[0], display_freq_limit[1])

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)


rms_sum = {}
spectra_sum = 0

for label in spectra:
    #plot fft vs. amplitude error from original 
    ax1.plot(fft_freq[label], 2 / n_samples * (np.abs(spectra[label]) / np.abs(reference_spectrum[:len(spectra[label])])), label=label)
    ax2.plot(fft_freq[label], np.angle(spectra[label]) - np.angle(reference_spectrum[:len(spectra[label])]))

    #deal with RMS at the same time:
    #square each value then sum them for RMS for each label
    error_array = (2 / n_samples * np.abs(spectra[label])**2)
    for num in error_array:
        spectra_sum += num
    rms_sum[label] = spectra_sum
    # print(rms_sum)

ax1.grid()
ax2.grid()
ax1.legend(loc = 'upper right')
ax1.set_ylabel('Amplitude Error')
ax2.set_ylabel('Phase Difference (rad)')
ax2.set_xlabel('Frequency (Hz)')
ax1.semilogy()
ax1.set_xlim(display_freq_limit[0], display_freq_limit[1])
ax1.set_ylim(top=1)


In [ ]:
#RMS values
#Right now only works if theres only 2 being compared
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.grid()

rms = {}

for label in rms_sum:
    rms[label] = math.sqrt(1/n_samples * rms_sum[label])

colors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:purple', 'tab:pink', 'tab:yellow']

ax1.bar(list(rms.keys()), list(rms.values()), width=0.8, color=colors[:len(spectra)])

ax1.set_ylabel('RMS Value')
ax1.set_title('Root Mean Square Error by Origin')


print(rms)
    